In [1]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt

# basedir = os.path.dirname(os.getcwd())
basedir = os.path.abspath(os.path.join(os.getcwd() ,"../"))
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')

sys.path.insert(1, _py)
import loads
import lia
import ray as rayt
import lad
import figures

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

%matplotlib qt


# Data structure

```
root
│   requirements.yml
│   readme.md  
│
└───data
│   └───test
│       │   s0100000.numpy
│       │   s0200000.numpy
│       │   ...
│       │   mesh.ply
│       │   scanner_pos.txt
│   
└───py
    │   loads.py
    │   lia.py
    │   lad.py
    │   ray.py
    │   figures.py

```

## Blensor output transformation

Most of the functions used in this chapter need:

``` Python
import loads
```

In order to get the `LIA` and hence the `LAD`, we need to segmentated the trees and the leaves.

First, we define the name of the directory where the Blensor output data is, in this particular case we will look for directory `test`. Pipeline will look for this directory inside the `data` directory.

In [2]:
mockname = 'test_kiwi'
# mockname = 'test'

Next, we convert Blensor output `txt` files that have fake `numpy` extension to real `npy`. This is done trhough function `loads.numpy2npy()` as shown below,

In [3]:
loads.numpy2npy(mockname)

s0500000.numpy done --> Number of beams: 62500
s0200000.numpy done --> Number of beams: 62500
s0400000.numpy done --> Number of beams: 62500
s0100000.numpy done --> Number of beams: 62500
s0600000.numpy done --> Number of beams: 62500
s0300000.numpy done --> Number of beams: 62500


```{note}
Transforming to `npy` reduce the size of files, besides is much faster to load than the Blensor `txt` output files.
```

The structure looks like,

```
root
|
└───data
    └───test
        │   s0100000.numpy
        │   s0200000.numpy
        │   ...
        │   mesh.ply
        │   scanner_pos.txt
        │   s0100000.npy
        │   s0200000.npy
        │   ...
```

## Tree and leaves segmentation

Now we create the module to segmentate trees. This will be tuned acordingly for each data set, so below module only works for this particular data set.

In [3]:
def segtree(df, leaves, show=False):

    trees = {}

    if show:
        plt.figure(figsize=(14, 8))

    # centres
    x, y = [0], [0]
    num = 0
    dx, dy = 2, 2
    # dx, dy = 5, 5

    for i in x:
        for j in y:
            
            keep = np.ones(len(df['x']), dtype=bool)
            keep &= (df['x'] < i+dx) & (df['x'] > i-dx)
            keep &= (df['y'] < j+dy) & (df['y'] > j-dy)

            trees['tree_%s' %(str(num))] = keep
            
            if show:
                plt.scatter(df['x'][leaves & keep], df['y'][leaves & keep], s=0.5, label=num)
                        
            num += 1

    if show:
        plt.legend()
    
    return trees


We segmentate the trees below,

In [4]:
# load data into a pandas data frame
df = loads.npy2pandas(mockname)
# extract leaves. Boolean array output
leaves = loads.extract_leaves(df, show=True)
# extract trees. Dictionary with boolean arrays output
trees = segtree(df, leaves, show=True)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


That's it! So what we just did? First, with function `loads.npy2pandas()` we load all the `npy` files into a pandas DataFrame (`DF`) and we add three more columns with the $x$, $y$, and $z$ positions of the sensors that are stored in file `scanner_pos.txt`. Then, since this is a mockup dataset, we can easily separate the leaves from everythin else in the point cloud (`PC`). We do this with function `loads.extract_leaves()` that requires the pandas `DF` as input. Finally, we invoke the above module to segmentate trees that requires the pandas `DF` as well.

outputs from this are:

- `df`: Pandas DF with the entire PC
- `leaves`: numpy boolean array of PC dimensions with True for Points concerning leaves only
- `trees`: python dictionary where each entry contains one tree in the form of boolena array with PC dimensions

Below piece of code shows an example of how to visualize the leaves points from one tree only, this will be know as the `Leaves Point Cloud` (LPC) and is shown in Fig. {numref}`lpc`.

Below piece of code shows an example of how to visualize the leaves points from one tree only.

In [7]:
# show the point cloud from leaves of firs tree only
keep = (trees['tree_0']) & (leaves)
loads.showPCfromDF(df[keep])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In the subsequent chapters we will be comparing our estimations with the *True* values using the `mesh.ply` file located in the root directory `test`. The following piece of code shows how we can visualize this mesh that requires importing the library `lad`. Fig. {numref}`mesh` show this mesh.

In [5]:
import lad
meshfile = lad.get_meshfile(mockname)
lad.see_mesh(meshfile)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


# `Leaf Inclination Angle` (LIA) estimation

## Intro

Most functions used in this chapter are in library:

``` Python
import lia
```

Outputs will be placed inside directory `lia`,

```
root
└───data
    └───test
        │   s0100000.numpy
        │   s0200000.numpy
        │   ...
        │   mesh.ply
        │   scanner_pos.txt
        │   s0100000.npy
        │   s0200000.npy
        │   ...
        └───lia
            │   angles_<treename>.npy
            │   weights_<treename>.npy
            │   leaf_angle_dist_<treename>.png
            │   leaf_angle_dist_height_<treename>.png
            │   bestfits_pars_treename>.png

```

## The method (`lia.leaf_angle()`)

The main function that computes the LIA is `lia.leaf_angle()` which uses a KDtree approximation. The steps are as follow: 

1. `Compute normals`: This method fits a plane based on the nearesth neighbors for each point and gets the normal of this plane.
    
2. `Compute zenith angles`: Then, using the dot product with get the angle with respect to the zenith (i.e. agains vector (0, 0, 1)) 
    
3. `Range correction`: The results angles run from $0 < \theta < 180$, however we require these to be in the range $0 < \theta < 90$ therefore we transfom those angles $> 90$ with relation:

```{math}
:label: angcorr
\theta_{L} = 180 - \theta
```

4. `Weights correction`: The resulting LIA is biased to PC density and completeness. In order to reduce this biases, we compute weights via voxelization,

```{math}
:label:
\eta_{i} = n_{i}/L^{3} \\
  \bar{\eta} = \frac{1}{N}\sum_{i=0}^{N} \eta{i}
```

where $n_{i}$ is the number of points within voxel $i$, $L$ is the voxel size, and $N$ is the total number of voxels, then $\eta_{i}$ is the volume density of voxel $i$, and $\bar{\eta}$ is the mean volume density.

The function `lia.leaf_angle()` has to be ran per tree and requieres 6 input parameters:

- `points`: $x$, $y$ and $z$ coordinates of the leaf point cloud (LPC).
- `mockname`: name of directory where the data is.
- `treename`: name/index of tree.
- `voxel_size_w`: voxel size for `weights correction` i.e. $L$.
- `kd3_sr`: KDtree searching radius for the nearest neighboors serch.
- `max_nn`: Maximum number of nearest neightbors to be considered.

This function returns a set of files inside directory `lia`:

- `angles_<treename>.npy`: LIA for the LPC. One file per tree.
- `weights_<treename>.npy`: LIA weights for the LPC. One file per tree.
- `leaf_angle_dist_<treename>.png`: Figure of LIA ($\theta_{L}$) distribution with `weights correction`. If `Truth` LIA available, this will be shown alongside. One figure per tree.
- `leaf_angle_dist_height_<treename>.png`: Top - Figure of LPC distribution accross different heights in terms of voxels $k$. Bottom - If `Truth` LIA available, $\theta_{L}^{truth} - \theta_{L}$. The different curves show this for different heights ($k$). One figure per tree.

## Look for best-fit `voxel_size_w`, `kd3_sr` and `max_nn` with `lia.bestfit_pars_la()`

If truth LIA available i.e. there's a mesh file `mesh.ply` in the `test` directory, then we will be able to run function `lia.bestfit_pars_la()` which essentialy runs `lia.leaf_angle()` for a range of values in `voxel_size_w`, `kd3_sr` and `max_nn` and find the best-fit for these three based on the minimal $\chi^{2}$ between the estimated LIA and the truth LIA.

`lia.bestfit_pars_la()` is as well ran per tree and requires only `points`, `mockname` and `treename`. It returns `bestfit_<treename>.npy` file that contains the `voxel_size_w`, `kd3_sr` and `max_nn` best-fit values per tree. it also returns a dictionary with the $\chi^{2}$ for each of these runs.

Using output dictionary from `bestfit_pars_la` we can run `bestfit_pars_la` to create figure `bestfits_pars_treename>.png` that shows the $\chi^{2}$ for all the ranges used in `voxel_size_w`, `kd3_sr` and `max_nn`.

```{admonition} To-Do
:class: important
Current LIA implementation works without `Truth` LIA, however, we need it to estimate the best-fits `voxel_size_w`, `kd3_sr` and `max_nn` parameters. We need to find the relation between these three and LPC that could rely on the LPC density, leaf size, leaf area, etc.
```

The piece of code bellow runs `lia.bestfit_pars_la()` and `lia.best_fit_pars_plot()` for each tree.


In [9]:
for key, val in trees.items():

    keep = (val) & (leaves) # take the LPC per tree
    df_ = df[['x', 'y', 'z']][keep]
    points = loads.DF2array(df_)
    res = lia.bestfit_pars_la(points, mockname, treename=key)
    lia.best_fit_pars_plot(res, key, mockname)


voxel_size_w 0.0001 DONE...
voxel_size_w 0.001 DONE...
voxel_size_w 0.01 DONE...
voxel_size_w 0.1 DONE...
voxel_size_w 1 DONE...
voxel_size_w BESTFIT:	 0.01
kd3_sr 0.001 DONE...
kd3_sr 0.01 DONE...
kd3_sr 0.1 DONE...
kd3_sr 1.0 DONE...
kd3_sr BESTFIT:	 0.1
max_nn 3 DONE...
max_nn 5 DONE...
max_nn 10 DONE...
max_nn 20 DONE...
max_nn 50 DONE...
max_nn 100 DONE...
max_nn BESTFIT:	 5


once we find the best-fit parameters we get figure `bestfits_pars_treename>.png` that is shown in Fig. {numref}`bestfits_pars`. we use these best fits to run `lia.leaf_angle()` and get the LIA and corresponding weigths per tree. The code that does that is shown below and in Fig. {numref}`lia_dist` we show `leaf_angle_dist_<treename>.png` and in Fig. {numref}`lia_dist_h` `leaf_angle_dist_height_<treename>.png`.

In [10]:
# load bestfit results
for key, val in trees.items():

    keep = (val) & (leaves)
    df_ = df[['x', 'y', 'z']][keep]
    points = loads.DF2array(df_)

    bestfit_file = os.path.join(_data, mockname, 'lia', 'bestfit_%s.npy' %(key))
    res = np.load(bestfit_file, allow_pickle=True)
    res = res.tolist()

    text = 'leaf area=%.2f \n %s=%.4f \n %s=%.4f \n %s=%.4f ' %(res['leafsize'], 'voxel_size_w', res['voxel_size_w_bestfit'],'kd3_sr', res['kd3_sr_bestfit'],'max_nn', res['max_nn_bestfit'])
    print(text)

    chi2 = lia.leaf_angle(points, mockname, key, res['voxel_size_w_bestfit'], 
                            res['kd3_sr_bestfit'], res['max_nn_bestfit'], save=True,
                                savefig=True, text=text)
                                

leaf area=0.01 
 voxel_size_w=0.0100 
 kd3_sr=0.1000 
 max_nn=5.0000 


# `Leaf Area Density` (LAD) estimation

## Intro

Most functions used in this chapter are in library:

``` Python
import lad
```

The LAD method implemented here uses the `Voxel 3D contact-bases frecuency` method first introduced by `HOSOI AND OMASA: VOXEL-BASED 3-D MODELING OF INDIVIDUAL TREES FOR ESTIMATING LAD`.

The model looks like:

```{math}
:label:
LAD(h, \Delta H) = \frac{1}{\Delta H} \sum_{k=m_{h}}^{m_{h}+\Delta H} l(k),
```

where,

```{math}
:label:
l(k) = \alpha(\theta)N(k) \\
    = \alpha(\theta) \cdot \frac{n_{I}(k)}{n_{I}(k) + n_{P}(k)}.
```

$l(k)$ is the `Leaf Area Index` (LAI) of the kth horizontal layer of the voxel array within a plant region, $\Delta H$ is the horizontal layer thickness, and $m_{h}$ and $m_{h}+\Delta H$ are the voxel coordinates on the vertical axis equivalent to height $h$ and $h+\Delta H$ in orthogonal coordinates ($h = \Delta k \times m_{h}$). The LAI of the kth horizontal layer $l(k)$ is the product of the contact frequency $N(k)$ of laser beams in the kth layer and the coefficient $\alpha(\theta)$, which corrects for leaf inclination at laser incident zenith angle $\theta$.

$n_{I}(k)$ is the number of voxels where the laser beams is intercepted by the kth layer, $n_{P}(k)$ is the number of voxels where the laser beams passed through the kth layer, and $n_{I}(k) + n_{P}(k)$ is the total number of voxels where the incident laser beams reach the kth layer.

Despite the complexity of this method, it requieres only one parameter, the `voxel_size`. We will introduce a second parameter, the `downsample` whose importance will be explained later. The main steps towards LAD estimation are:

1. Computing $n_{P}(k)$
2. Computing $n_{I}(k)$
3. Computing $\alpha(\theta)$
4. Estimate LAD

For this example, we will usea a `voxel_size` = 0.2 and `downsample` = 0.05 which means that we downsample our whole data to only $5\%$. A reminder that as well as in for the LIA, the following process is per tree.

In [11]:
downsample = 0.1
voxel_size = 0.2
# to check everything looks fine
show = True
sample = 5

## Computing $n_{P}(k)$

Seeing where the beam pass through in the voxelize `Plant Region` (PR) is a tipycal ray tracing problem and it's reduced to see whether the ray hit or not an axis align bounding box (AABB).

The below module grabs the requiered downsample percentge of the data with a random subsample and if it's the first time we ran this, it will create the directory `lad_<downsmple>`. All the subsequent results will be stored inside this directory. The first time a particular downsample is ran, it will store the file `inds.npy` containing a boolean array with size of the pandas DF where True being the selected random subsample requested. If we change the `voxel_size` but not the `downsample`, then the below module will look first for the `inds.npy` instead of searching for another random subsample, this to maintain uniformity between different voxels sizes approaches.

The function `main` does the magic here, it has to be ran per tree and requires 6 input parameters:

- `points`: $x$, $y$ and $z$ coordinates from the downsample data in the form of numpy array.
- `sensors`: $x$, $y$ and $z$ coordinates of sensor responsible from each point in `points` parameter above.
- `pointsPR`: `points` above filtered to the LPC.
- `voxel_size`: Voxel Size.
- `resdir`: Name of output directory for the specific `downsample`.
- `treename`: Name/index of tree.

```{note}
`pointsPR` is require to get the same voxelization dimensions as in $n_{I}$.
```
This function returns two files:

- `m3s_<treename>_<voxel_size>.npy`: numpy boolean 3D-array with number of voxels dimensions. True if a beam hit the voxel.
- `m3count_<treename>_<voxel_size>.npy`: numpy 3D-array with number of voxels dimensions. Each entry contains the number of beams that passed trhough that voxel.

```{admonition} To-Do
:class: important
Note that this is the slowest module of the entire pipeline, taking up to 5 minutes for a sample of 10,000 beams. This can be improved easlily if binding with a `C++` ray AABB module instead.
```

Below we show the piece of code that computes this,

In [6]:

POINTS = loads.DF2array(df[['x', 'y', 'z']])
SENSORS = loads.DF2array(df[['sx', 'sy', 'sz']])

def get_rays(downsample, voxel_size, sample=None, show=False):

    if downsample is not None:

        resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
        if not os.path.exists(resdir):
            os.makedirs(resdir)

        outdir = os.path.join(resdir, 'inds.npy')
        if os.path.exists(outdir):
            print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

            inds = np.load(outdir)

            points = POINTS[inds]
            sensors = SENSORS[inds]

        else:

            print('inds not been created yet for donwnsample of %.3f' %(downsample))
            idx = np.random.randint(0, len(df), int(len(df) * downsample))
            inds = np.zeros(len(df), dtype=bool)
            inds[idx] = True

            points = POINTS[inds]
            sensors = SENSORS[inds]

            np.save(outdir, inds)

    else:

        resdir = os.path.join(_data, mockname, 'lad')
        if not os.path.exists(resdir):
            os.makedirs(resdir)

    if sample is not None:

        idx = np.random.randint(0, len(df), int(sample))
        points = POINTS[idx]
        sensors = SENSORS[idx]

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        m3s = rayt.main(points, sensors, pointsPR, voxel_size, resdir, key, show=show)

## Computing $n_{I}(k)$

The $n_{I}$ per voxel is computed in function `lad.compute_attributes()`. It essentialy voxelize the LPC to get the PR dimensions (which have to be the same as in $n_{P}$). Then, for a numpy boolean 3D-array with voxelize PR dimensions, we fill it with True if there's a point in the voxel.

This function looks for previous `m3s_<treename>_<voxel_size>.npy` result and get the attributes in the form of the same size numpy 3D-array (`m3att`). The attributes are:

- 1 if any LPC in that voxel
- 2 if any beam pass trhough that  voxel
- 3 if none of previous

It requires 4 input parameters `pointsPR`, `resdir`, `voxel_size`, `treename` which were defined in section {ref}`sec:np`. It returns the attributes numpy 3D-array.

## Computing $\alpha(\theta)$

$\alpha(\theta)$ is expressed in terms of $G(\theta)$, 

```{math}
:label:
\alpha(\theta) = \frac{\cos(\theta)}{G(\theta)},
```

where $G(\theta)$ is the mean projection of a unit leaf area on a plane perpendicular to the direction of the laser beam. This quantity is determined with the assumption that leaves are positioned symmetrically with respect to the azimuth anc can be represented as:

```{math}
:label:
G(\theta) =  \sum_{q=1}^{T_{q}} g(q) S(\theta, \theta_{L}(q))
```

where $S(\theta, \theta_{L}(q))$ is expresed in terms of the leaf inclination angle (LIA) $\theta_{L}$ (the zenith angle of the normal to the leaf surface), and $\theta$ is the laser-beam incident zenith angle:

```{math}
:label:
S(\theta, \theta_{L}) = \cos\theta \cos \theta_{L}, \hspace{.5cm} \textrm{for } \theta \leq \pi/2 - \theta_{L}
```

```{math}
:label:
S(\theta, \theta_{L}) = \cos\theta \cos \theta_{L} \left[ 1 + \frac{2}{\pi}(\tan x - x) \right], \hspace{.5cm} \textrm{for } \theta \gt \pi/2 - \theta_{L}
```

```{math}
:label:
x = \cos^{-1}\left( \cot \theta \cot \theta_{L} \right).
```

Here $q$ is the leaf-inclination-angle class and Tq is the total number of leaf-inclination-angle classes. Thus, if there are $18$ leaf-inclination-angle classes from $0◦$ to $90◦$ ($Tq = 18$), then each class consists of a $5◦$ interval. For example, $q = 1$, $q = 9$, and $q = 16$ include the angles from $0◦$ to $4◦$, $40◦$ to $44◦$, and $75◦$ to $79◦$, respectively. $g(q)$ is the distribution of the leaf-inclination-angle class $q$, which is a ratio of the leaf area belonging to class $q$ to total leaf area; $θ_{L}(q)$ is the midpoint angle of class $q$, which is the leaf-inclination angle used to represent class $q$.

This process is done trhough function `lad.Gtheta()`. In function `lad.alpha_k()` we compute $\alpha(\theta)$ for the median of $\theta$, the Beam Inclination Angles (BIA) with respect to zenith, in the Kth layer. We made use of the files `angles_<treename>.npy` and `weights_<treename>.npy` we store previously in the directory `lia` to get $g(q)$. The function `lad.alpha_k()` create three figures inside the `figures` directory:

- `alphas_<treename>_<voxel_size>.png`
- `bia_<treename>_<voxel_size>.png`
- `bia_per_k_<treename>_<voxel_size>.png`

Examples of this three figures can be found in Figures {numref}`alphasplot`, {numref}`biaplot`, and {numref}`biakplot` for a `downsample` of $5 \%$.

## Estimate LAD

Now that we have $\alpha(\theta)$ in the Kth layer (i.e. $\alpha(\theta, k)$), we can compute the LAI and therefore the LAD. We do this in function `lad.get_LADS()` which requires 4 input parameters:

- `m3att`: The numpy 3D-array attributes we derive in section {ref}`sec:ni`
- `voxel_size`: Voxel Size.
- `kbins`: $\Delta H$ in lengths if K.
- `alphas_k`: `lad.alpha_k()` function output.

This returns a numpy 2D-array with the height and LAD for the corresponding height with zero being the bottom of the PR.

Finally, with function `figures.plot_lads()` we plot LAD as a function of height for:

1. Using correction of $\alpha(\theta, K)$ taking the median of $\theta$ in the Kth layer.
2. Without $\alpha(\theta, K)$ correction
3. Truth LAD from mesh file.

The piece of code below we show all the above mentioned steps plus other minor steps. The output figure is saved in directory `figures` with name `LAD_<treename>_<voxel_size>.png` and shown in Fig. {numref}`ladplot`.

In [7]:
def runall(downsample, voxel_size, kbins=None):
    
    if downsample is not None:
        resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
        inds_file = os.path.join(resdir, 'inds.npy')
        inds = np.load(inds_file)
        print('downsample:', downsample)
    else:
        inds = np.ones(len(df), dtype=bool)
        resdir = os.path.join(_data, mockname, 'lad')

    isfigures = os.path.join(resdir, 'figures')
    if not os.path.exists(isfigures):
        os.makedirs(isfigures)

    print('voxel_size:', voxel_size)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        sensorsPR = SENSORS[inPR]

        m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, key)

        _,_,_, m3scount = lad.density_counts(pointsPR, voxel_size)

        # get in down sample boolean array for LPC size
        inds_ = inds[(val) & (leaves)]
        lias, ws = lad.downsample_lia(mockname, key, inds_)
        voxk = lad.get_voxk(pointsPR, voxel_size)
        bia = lad.get_bia(pointsPR, sensorsPR)
        meshfile = lad.get_meshfile(mockname)

        figext = '%s_%s' %(key, str(voxel_size))
        # figext = None
        alphas_k = lad.alpha_k(bia, voxk, lias, ws, resdir, meshfile, figext=figext, 
                                klia=False, use_true_lia=True)

        kmax = m3att.shape[2]
        if kbins is None:
            kbins = int(kmax/15)
        print(kbins)

        # Attribute 2 counts per voxel
        outdir_count = os.path.join(resdir, 'm3count_%s_%s.npy' %(key, str(voxel_size)))
        
        # print('******* outdir_count', outdir_count)

        
        # lads_min = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,2], 1)
        # lads_max = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,4], 1)
        lads_mid = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1)

        # try:
        #     m3pcount = np.load(outdir_count)
        #     lads_mid_w = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1, m3scount=m3scount, m3pcount=m3pcount)
        #     lads_mid_counts = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6], 1, m3scount=m3scount, usecounts=True, m3pcount=m3pcount)
        # except:
        #     print('no %s' %(outdir_count))

        lads_0 = lad.get_LADS(m3att, voxel_size, kbins, alphas_k[:,6]*0+1, 1.0)
        lads_mesh = lad.get_LADS_mesh(meshfile, voxel_size, kbins, kmax)

        # lads = {'Truth':lads_mesh, 'Correction Mean':lads_mid, 'No Correction':lads_0, 'Correction Weights':lads_mid_w}#, 'Correction counts':lads_mid_counts}
        lads = {'Truth':lads_mesh, 'Correction Mean':lads_mid, 'No Correction':lads_0}
        clai = lad.get_clai(m3att, alphas_k)
        attributes_file = os.path.join(resdir, 'm3s_%s_%s.npy' %(key, str(voxel_size)))
        if os.path.isfile(attributes_file):
            RT = 'Y'
        else:
            RT = 'N'
            
        text = {'tree':key, 'VS':voxel_size, 'DS':downsample, 'RT':RT, 'CLAI':np.round(clai, 3)}
        txt = []
        for key, val in text.items():
            txt.append('%s=%s \n' %(key, str(val)))
        text = (' ').join(txt)

        savefig = os.path.join(resdir, 'figures','LAD_%s.png' %(figext))
        figures.plot_lads(lads, text, savefig=savefig)

    # return m3pcounts, m3scount



In [8]:
downsample = 0.2
voxel_size = 0.1

runall(downsample, voxel_size, kbins=1)

downsample: 0.2
voxel_size: 0.1
max --> [30, 34, 10]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (31, 35, 11)
ray tracker voxel dimensions: 	 (31, 35, 11)
Number of voxels ocupied by points cloud: 	 2418
Number of voxels ocupied by beam points cloud: 	 7506
Total number of voxels in plant regions: 	 11935
Number of voxels with attribute 1: 	 2418
Number of voxels with attribute 2: 	 7506
Number of voxels with attribute 3: 	 2011
max --> [30, 34, 10]
min --> [0, 0, 0]
1


In [16]:
m3scount.shape

(31, 35, 11)

In [17]:
m3pcounts.shape

(31, 35, 11)

In [18]:
a = m3scount[:,:,1].sum()
b = m3pcounts[:,:,1].sum()
print(a,b)

212 74791


In [51]:
res = []
for k in range(m3scount.shape[2]):

    a = (m3scount[:,:,k] != 0).sum()
    b = (m3pcounts[:,:,k] != 0).sum()
    lai = a / (a + b)

    a1 = m3scount[:,:,k].sum()
    b1 = m3pcounts[:,:,k].sum()
    lai1 = a1 / (a1 + b1)

    print(k, lai1)

    res.append([k, lai, lai1])

res = np.array(res)
res.T[1]

plt.plot(res.T[0], res.T[1], marker='o', label='voxel count')
plt.plot(res.T[0], res.T[2], marker='o', label='point count')
plt.legend()
plt.show()

0 0.00010954252303816187
1 0.0028265536045224858
2 0.009177660739427438
3 0.03327934598179855
4 0.08579118819726565
5 0.10609558426248833
6 0.07914677003187354
7 0.03337338744416986
8 0.01074671533674175
9 0.0011161103554113913
10 0.00017818324364776735


In [138]:
downsample = 0.5
voxel_size = 0.1
# to check everything looks fine
show = False
sample = None

get_rays(downsample, voxel_size, sample, show)

inds not been created yet for donwnsample of 0.500
max --> [30, 34, 10]
min --> [0, 0, 0]


1107it [00:12, 91.66it/s] 


KeyboardInterrupt: 

In [15]:
for DS in [0.2, 0.4, 0.6]:
    for VS in [0.05, 0.08, 0.1]:

        get_rays(DS, VS)

inds file already exists for donwnsample of 0.200 at /Users/omar/projects/planttech/data/test/lad_0.2/inds.npy
max --> [188, 182, 116]
min --> [0, 0, 0]


28463it [27:36, 17.18it/s]


tot vox: 	 4046679
voxels hitted: 	 1963773
Percentage of voxels hitted by beam: 0.49
voxels hitted (OLD): 	 2052638
Percentage of voxels hitted by beam (OLD): 0.51
inds file already exists for donwnsample of 0.200 at /Users/omar/projects/planttech/data/test/lad_0.2/inds.npy
max --> [117, 113, 73]
min --> [0, 0, 0]


28463it [15:20, 30.91it/s]


tot vox: 	 995448
voxels hitted: 	 703584
Percentage of voxels hitted by beam: 0.71
voxels hitted (OLD): 	 763198
Percentage of voxels hitted by beam (OLD): 0.77
inds file already exists for donwnsample of 0.200 at /Users/omar/projects/planttech/data/test/lad_0.2/inds.npy
max --> [94, 91, 58]
min --> [0, 0, 0]


...

tot vox: 	 4195800
voxels hitted: 	 3086910
Percentage of voxels hitted by beam: 0.74
voxels hitted (OLD): 	 3261074
Percentage of voxels hitted by beam (OLD): 0.78
inds file already exists for donwnsample of 0.600 at /Users/omar/projects/planttech/data/test/lad_0.6/inds.npy
max --> [118, 115, 74]
min --> [0, 0, 0]


70990it [42:27, 27.87it/s]


tot vox: 	 1035300
voxels hitted: 	 850847
Percentage of voxels hitted by beam: 0.82
voxels hitted (OLD): 	 927935
Percentage of voxels hitted by beam (OLD): 0.90
inds file already exists for donwnsample of 0.600 at /Users/omar/projects/planttech/data/test/lad_0.6/inds.npy
max --> [94, 92, 60]
min --> [0, 0, 0]


70990it [38:50, 30.46it/s]


tot vox: 	 538935
voxels hitted: 	 446752
Percentage of voxels hitted by beam: 0.83
voxels hitted (OLD): 	 498311
Percentage of voxels hitted by beam (OLD): 0.92


In [57]:
for DS in [0.2, 0.4, 0.6]:
    for VS in [0.05, 0.08, 0.1]:

        runall(downsample=DS, voxel_size=VS, kbins=1)

downsample: 0.2
voxel_size: 0.05
max --> [60, 68, 20]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (61, 69, 21)
ray tracker voxel dimensions: 	 (61, 69, 21)
Number of voxels ocupied by points cloud: 	 8115
Number of voxels ocupied by beam points cloud: 	 63021
Total number of voxels in plant regions: 	 88389
Number of voxels with attribute 1: 	 8115
Number of voxels with attribute 2: 	 63021
Number of voxels with attribute 3: 	 17253
max --> [60, 68, 20]
min --> [0, 0, 0]
8115 5979 2136
Counts 29756
Density overal= 2693.19
Mean density= 27694.86
Mean density considering all voxels= 2493.16
Median density= 24000.00
1
downsample: 0.2
voxel_size: 0.08
max --> [38, 42, 12]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (39, 43, 13)
ray tracker voxel dimensions: 	 (39, 43, 13)
Number of voxels ocupied by points cloud: 	 3708
Number of voxels ocupied by beam points cloud: 	 14332
Total number of voxels in plant regions: 	 21801
Number of voxels with attribute 1: 	 3708
Number of voxels with 

## Counts within voxels

In [9]:
def isinds(resdir, downsample):

    outdir = os.path.join(resdir, 'inds.npy')
    if os.path.exists(outdir):
        print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

        inds = np.load(outdir)

        points = POINTS[inds]
        sensors = SENSORS[inds]

    else:

        print('inds not been created yet for donwnsample of %.3f' %(downsample))
        idx = np.random.randint(0, len(df), int(len(df) * downsample))
        
        inds = np.zeros(len(df), dtype=bool)
        inds[idx] = True

        points = POINTS[inds]
        sensors = SENSORS[inds]

        np.save(outdir, inds)

    return inds

def isinds_vs(resdir, voxel_size):

    outdir = os.path.join(resdir, 'inds_vs.npy')
    if os.path.exists(outdir):
        print('inds file already exists for donwnsample of %.3f at %s' %(voxel_size, outdir))

        inds = np.load(outdir)

        points = POINTS[inds]
        sensors = SENSORS[inds]

    else:

        print('inds not been created yet for donwnsample of %.3f' %(voxel_size))
        # idx = np.random.randint(0, len(df), int(len(df) * downsample))

        pcd = loads.points2pcd(POINTS)
        minb = pcd.get_min_bound()
        maxb = pcd.get_max_bound()
        voxel = o3d.geometry.PointCloud.voxel_down_sample_and_trace(pcd, voxel_size=voxel_size, min_bound=minb, max_bound=maxb)
        idxDS = [i[0] for i in voxel[2]]
        
        inds = np.zeros(len(df), dtype=bool)
        inds[idxDS] = True

        points = POINTS[inds]
        sensors = SENSORS[inds]

        np.save(outdir, inds)

    return inds

def restmp(downsample, voxel_size, show=True, indsvs=False):

    resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)

    if indsvs:
        inds = isinds_vs(resdir, voxel_size)
        print('Total:',len(df))
        print('Downsampling:',np.sum(inds))
    else:    
        inds = isinds(resdir, downsample)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        sensorsPR = SENSORS[inPR]

        density_overall, density, counts, m3scount = lad.density_counts(pointsPR, voxel_size)
        
        dens_ratio = density / density_overall
        # print('Mean density ration: %.2f' %(np.mean(dens_ratio)))
        # print('Median density ration: %.2f' %(np.median(dens_ratio)))

        if show:

            fig = plt.figure(figsize=(10,6))
            plt.hist(counts, 25, align='left')
            plt.axvline(np.mean(counts), color='k', label='Mean counts = %.2f' %(np.mean(counts)))
            plt.xlabel(r'$n_{i}$', size=20)
            plt.legend()
            plt.show()

    return np.mean(counts)


In [10]:
def isinds(resdir, downsample):

    outdir = os.path.join(resdir, 'inds.npy')
    if os.path.exists(outdir):
        print('inds file already exists for donwnsample of %.3f at %s' %(downsample, outdir))

        inds = np.load(outdir)

        points = POINTS[inds]
        sensors = SENSORS[inds]

    else:

        print('inds not been created yet for donwnsample of %.3f' %(downsample))
        idx = np.random.randint(0, len(df), int(len(df) * downsample))
        inds = np.zeros(len(df), dtype=bool)
        inds[idx] = True

        points = POINTS[inds]
        sensors = SENSORS[inds]

        np.save(outdir, inds)

    return inds

def restmp(downsample, voxel_size, show=True):

    resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)
    inds = isinds(resdir, downsample)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]
        sensorsPR = SENSORS[inPR]

        density_overall, density, counts, m3scount = lad.density_counts(pointsPR, voxel_size)
        
        # dens_ratio = density / density_overall
        # print('Mean density ration: %.2f' %(np.mean(dens_ratio)))
        # print('Median density ration: %.2f' %(np.median(dens_ratio)))

        if show:

            fig = plt.figure(figsize=(10,6))
            plt.hist(counts, 25, align='left')
            plt.axvline(np.mean(counts), color='k', label='Mean counts = %.2f' %(np.mean(counts)))
            plt.xlabel(r'$n_{i}$', size=20)
            plt.legend()
            plt.show()

    return np.mean(counts)


def ds2vs_first(DS, PCR):

    N = np.cbrt(1/DS)
    VS = N * PCR
    
    return VS

def ds2vs(downsample, points, mean_counts=0, step=0.01, bounds=(1.25, 1.3)):

    # nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(points)
    # distances, indices = nbrs.kneighbors(points)

    minb, maxb = bounds

    # dist = distances[:, 1]
    # dmin, dmax = np.percentile(dist, (0, 99.7))
    # keep = (dist >= dmin) & (dist <= dmax)
    # dist = dist[keep]
    # PCR = np.round(np.mean(dist), 4)
    # print('PCR first guess: %.3f' %(PCR))
    PCR = 0.01

    # first guess for voxel size
    voxel_size = ds2vs_first(downsample, PCR)

    resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
    if not os.path.exists(resdir):
        os.makedirs(resdir)
    inds = isinds(resdir, downsample)

    for key, val in trees.items():

        inPR = (val) & (leaves) & (inds)
        pointsPR = POINTS[inPR]

        j = 0

        while (mean_counts < minb) or (mean_counts > maxb):

            # mean_counts = restmp(downsample, voxel_size, show=False)
            density_overall, density, counts, m3scount = lad.density_counts(pointsPR, voxel_size)
            mean_counts = np.mean(counts)
            
            print(mean_counts, voxel_size, step)

            if mean_counts < minb:
                voxel_size += step

            elif mean_counts > maxb:
                voxel_size -= step

            if np.abs(mean_counts - (maxb + minb)/2) < 0.1:
                step /= 2

    return voxel_size


In [11]:
inPR = (trees['tree_0']) & (leaves)
voxels = {}

for DS in [0.05, 0.1, 0.2]:

    VS = ds2vs(DS, POINTS[inPR], mean_counts=0, step=0.01, bounds=(1.25, 1.3))
    VS = np.round(VS, 3)
    voxels[DS] = VS

inds file already exists for donwnsample of 0.050 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.05/inds.npy
max --> [109, 125, 36]
min --> [0, 0, 0]
1.2129175946547883 0.02714417616594907 0.01
max --> [80, 91, 26]
min --> [0, 0, 0]
1.394741335154516 0.03714417616594907 0.005
max --> [92, 105, 30]
min --> [0, 0, 0]
1.3082959641255605 0.03214417616594907 0.005
max --> [109, 125, 36]
min --> [0, 0, 0]
1.2129175946547883 0.02714417616594907 0.0025
max --> [100, 114, 33]
min --> [0, 0, 0]
1.2587057010785825 0.029644176165949068 0.00125
inds file already exists for donwnsample of 0.100 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.1/inds.npy
max --> [140, 156, 46]
min --> [0, 0, 0]
1.253128237825775 0.02154434690031884 0.01
inds file already exists for donwnsample of 0.200 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.2/inds.npy
max --> [176, 198, 58]
min --> [0, 0, 0]
1.30228894043503 0.01709975946676697 0.01


In [209]:
mean_counts = restmp(downsample=0.1, voxel_size=0.1, show=True, indsvs=True)
print(mean_counts)

inds not been created yet for donwnsample of 0.100
Total: 375000
Downsampling: 35497
max --> [30, 33, 10]
min --> [0, 0, 0]
1.5332926084300549


## Ni and Np variation

In [72]:
results = []

for downsample in [0.01, 0.1, 0.2, 0.4, 0.6]:
    for voxel_size in [0.05]:

        if downsample is not None:
            resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
            inds_file = os.path.join(resdir, 'inds.npy')
            inds = np.load(inds_file)
            print('downsample:', downsample)
        else:
            inds = np.ones(len(df), dtype=bool)
            resdir = os.path.join(_data, mockname, 'lad')

        isfigures = os.path.join(resdir, 'figures')
        if not os.path.exists(isfigures):
            os.makedirs(isfigures)

        print('voxel_size:', voxel_size)

        for key, val in trees.items():

            inPR = (val) & (leaves) & (inds)
            pointsPR = POINTS[inPR]
            sensorsPR = SENSORS[inPR]

            m3att = lad.compute_attributes(pointsPR, resdir, voxel_size, key)

            results.append([downsample, voxel_size, (m3att == 1).sum(), (m3att == 2).sum(), (m3att == 3).sum()])

results = np.array(results)

total = np.array(np.sum(results[:,2:5], axis=1))
nI0 = results[:,2] / total
nP0 = results[:,3] / total
n00 = results[:,4] / total

nI = results[:,2]
nP = results[:,3]
n0 = results[:,4]


fig, (a0, a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [2, 1]}, figsize=(10,22))
colors = plt.cm.jet(np.linspace(0,1,5))

for num, voxel_size in enumerate([0.05]):

    keep = (results[:,1] == voxel_size)
    downsample = results[:,0][keep]
    a0.plot(downsample*100, nP0[keep], marker='*', color=colors[num], label='%s' %(voxel_size))
    a0.plot(downsample*100, nI0[keep], marker='*', ls='--', color=colors[num])
    a0.plot(downsample*100, n00[keep], marker='*', ls=':', color=colors[num])
    
    a1.plot(downsample*100, nI[keep]/(nI[keep]+nP[keep]), marker='*', ls='-', color=colors[num])

text = '$n_{P}(k)$: Solid \n $n_{I}(k)$: Dashed \n $n_{0}(k)$: Dotted'
props = dict(boxstyle='round', facecolor='green', alpha=0.3)
a0.text(20, 0.5, text, fontsize=14, bbox=props)
a0.legend(title='Voxel Size')
a0.set_ylabel(r'$N$', size=20)

# a1.axhline(0.035, ls='--', c='k', lw=2)
a1.set_xlabel(r'Downsample (%)', size=20)
a1.set_ylabel(r'$n_{I}/(n_{I}+n_{P})$', size=20)
    


downsample: 0.01
voxel_size: 0.05
max --> [60, 68, 18]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (61, 69, 19)
ray tracker voxel dimensions: 	 (61, 69, 19)
Number of voxels ocupied by points cloud: 	 1475
Number of voxels ocupied by beam points cloud: 	 40779
Total number of voxels in plant regions: 	 79971
Number of voxels with attribute 1: 	 1475
Number of voxels with attribute 2: 	 40779
Number of voxels with attribute 3: 	 37717
downsample: 0.1
voxel_size: 0.05
max --> [60, 67, 20]
min --> [0, 0, 0]
No ray tracing for tree tree_0 and voxel size 0.050
foliage voxel dimensions: 	 (61, 68, 21)
ray tracker voxel dimensions: 	 (61, 68, 21)
Number of voxels ocupied by points cloud: 	 6528
Number of voxels ocupied by beam points cloud: 	 87108
Total number of voxels in plant regions: 	 87108
Number of voxels with attribute 1: 	 6528
Number of voxels with attribute 2: 	 80580
Number of voxels with attribute 3: 	 0
downsample: 0.2
voxel_size: 0.05
max --> [60, 68, 20]
min --> [0, 0, 0]
f

Text(0, 0.5, '$n_{I}/(n_{I}+n_{P})$')

# DEV

In [137]:
POINTS[leaves].shape

(165062, 3)

In [201]:
import open3d as o3d

pcd = loads.points2pcd(POINTS[leaves])
minb = pcd.get_min_bound()
maxb = pcd.get_max_bound()
voxel = o3d.geometry.PointCloud.voxel_down_sample_and_trace(pcd, voxel_size=0.5, min_bound=minb, max_bound=maxb)
idxDS = [i[0] for i in voxel[2]]

In [203]:
pointslist = [POINTS[leaves], POINTS[leaves][idxDS]]
colours = [[0,0,1], [1,0,0]]
loads.showPCDS(pointslist, colours)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [131]:
DS = 0.1
VS = ds2vs(DS, POINTS[inPR], mean_counts=0, step=0.02)
VS = np.round(VS, 3)
print(VS)
get_rays(DS, VS)

PCR first guess: 0.006
inds file already exists for donwnsample of 0.100 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.1/inds.npy
max --> [250, 279, 82]
min --> [0, 0, 0]
1.068574147070817 0.012064834264178549 0.02
inds file already exists for donwnsample of 0.100 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.1/inds.npy
max --> [94, 105, 31]
min --> [0, 0, 0]
1.5718284514645606 0.03206483426417855 0.01
inds file already exists for donwnsample of 0.100 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.1/inds.npy
max --> [137, 153, 45]
min --> [0, 0, 0]
1.2706481331824795 0.022064834264178546 0.005
0.022
inds file already exists for donwnsample of 0.100 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.1/inds.npy
max --> [137, 153, 45]
min --> [0, 0, 0]


35680it [32:09, 18.49it/s]


tot vox: 	 977592
voxels hitted: 	 657004
Percentage of voxels hitted by beam: 0.67
voxels hitted (OLD): 	 708817
Percentage of voxels hitted by beam (OLD): 0.73


In [132]:
runall(downsample=0.1, voxel_size=0.022, kbins=2)

downsample: 0.1
voxel_size: 0.022
max --> [137, 153, 45]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (138, 154, 46)
ray tracker voxel dimensions: 	 (138, 154, 46)
Number of voxels ocupied by points cloud: 	 12429
Number of voxels ocupied by beam points cloud: 	 657004
Total number of voxels in plant regions: 	 977592
Number of voxels with attribute 1: 	 12429
Number of voxels with attribute 2: 	 657004
Number of voxels with attribute 3: 	 308159
max --> [137, 153, 45]
min --> [0, 0, 0]
2


In [133]:
from sklearn.neighbors import NearestNeighbors

# # keep = trees['tree_0']
# # print(np.sum(keep))
# downsample = 0.05

# resdir = os.path.join(_data, mockname, 'lad_%s' %(str(downsample)))
# inds_file = os.path.join(resdir, 'inds.npy')
# inds = np.load(inds_file)

# inPR = (trees['tree_0']) & (leaves) & (inds)
# pointsPR = POINTS[inPR]

nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(POINTS)
distances, indices = nbrs.kneighbors(POINTS)

In [135]:
# 
    
dist = distances[:, 1]
dmin, dmax = np.percentile(dist, (0, 99.7))
keep = (dist >= dmin) & (dist <= dmax)
dist = dist[keep]
mean = np.round(np.mean(dist), 3)
median = np.round(np.median(dist), 3)
print(dmin, dmax)

plt.figure(figsize=(10, 6))

bins = np.linspace(dmin, dmax, 80)
plt.hist(dist, bins, density=True)
plt.axvline(mean, ls='--', c='k', label='Mean = %.3f' %(mean))
plt.axvline(median, ls='--', c='g', label='Median = %.3f' %(median))
plt.xlabel('voxel size', size=15)

plt.legend()
plt.show()

0.0 0.09078427403044305


In [128]:
def ds2vs_first(DS, PCR):

    N = np.cbrt(1/DS)
    VS = N * PCR
    
    return VS

def ds2vs(downsample, points, mean_counts=0, step=0.01):

    nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(points)
    distances, indices = nbrs.kneighbors(points)

    minb, maxb = 1.25, 1.3

    dist = distances[:, 1]
    dmin, dmax = np.percentile(dist, (0, 99.7))
    keep = (dist >= dmin) & (dist <= dmax)
    dist = dist[keep]
    PCR = np.round(np.mean(dist), 4)
    print('PCR first guess: %.3f' %(PCR))

    # first guess for voxel size
    voxel_size = ds2vs_first(downsample, PCR)

    while (mean_counts < minb) or (mean_counts > maxb):

        mean_counts = restmp(downsample, voxel_size, show=False)
        print(mean_counts, voxel_size, step)

        if mean_counts < minb:
            voxel_size += step
            step /= 2

        elif mean_counts > maxb:

            voxel_size -= step
            step /= 2

    return voxel_size

In [129]:
inPR = (trees['tree_0']) & (leaves)
voxels = {}

for DS in [0.05, 0.1]:

    VS = ds2vs(DS, POINTS[inPR], mean_counts=0, step=0.02)
    VS = np.round(VS, 3)
    voxels[DS] = VS


PCR first guess: 0.006
inds file already exists for donwnsample of 0.050 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.05/inds.npy
max --> [195, 222, 64]
min --> [0, 0, 0]
1.0599455040871935 0.015200738652931478 0.02
inds file already exists for donwnsample of 0.050 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.05/inds.npy
max --> [84, 96, 27]
min --> [0, 0, 0]
1.3651403743315509 0.03520073865293148 0.01
inds file already exists for donwnsample of 0.050 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.05/inds.npy
max --> [118, 134, 38]
min --> [0, 0, 0]
1.1859756097560976 0.02520073865293148 0.005
inds file already exists for donwnsample of 0.050 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.05/inds.npy
max --> [98, 112, 32]
min --> [0, 0, 0]
1.2643553629469122 0.03020073865293148 0.0025
PCR first guess: 0.006
inds file already exists for donwnsample of 0.100 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.1/inds.npy
max --> [250, 279, 82]
min --

In [130]:
for key, val in voxels.items():
    print(key, val)

0.05 0.03
0.1 0.022


In [62]:
for DS in [0.01, 0.05, 0.1, 0.2, 0.4, 0.6]:

    VS = ds2vs(DS, 0.014)
    print(DS, VS)

0.01 0.0649822436705789
0.05 0.038001846632328695
0.1 0.030162085660446373
0.2 0.02393966325347376
0.4 0.019000923316164348
0.6 0.016598835420953625


In [136]:
counts, m3scount = restmp(downsample=0.05, voxel_size=0.03)

inds file already exists for donwnsample of 0.050 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.05/inds.npy
max --> [99, 113, 32]
min --> [0, 0, 0]


TypeError: cannot unpack non-iterable numpy.float64 object

In [73]:
for DS in [0.2]:

    VS = ds2vs(DS, 0.014)
    VS = np.round(VS, 3)
    print(DS, VS)

    get_rays(DS, VS)

0.2 0.024
inds file already exists for donwnsample of 0.200 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.2/inds.npy
max --> [125, 141, 42]
min --> [0, 0, 0]


67879it [59:00, 19.17it/s]


tot vox: 	 769356
voxels hitted: 	 588106
Percentage of voxels hitted by beam: 0.76
voxels hitted (OLD): 	 637916
Percentage of voxels hitted by beam (OLD): 0.83


In [64]:
for DS in [0.01, 0.05, 0.1]:

    VS = ds2vs(DS, 0.014)
    VS = np.round(VS, 3)
    print(DS, VS)

    get_rays(DS, VS)

0.01 0.065
inds not been created yet for donwnsample of 0.010
max --> [46, 52, 14]
min --> [0, 0, 0]


3732it [01:20, 46.28it/s] 


tot vox: 	 37365
voxels hitted: 	 22978
Percentage of voxels hitted by beam: 0.61
voxels hitted (OLD): 	 26462
Percentage of voxels hitted by beam (OLD): 0.71
0.05 0.038
inds file already exists for donwnsample of 0.050 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.05/inds.npy
max --> [78, 89, 25]
min --> [0, 0, 0]


18298it [12:28, 24.45it/s]


tot vox: 	 184860
voxels hitted: 	 130127
Percentage of voxels hitted by beam: 0.70
voxels hitted (OLD): 	 146928
Percentage of voxels hitted by beam (OLD): 0.79
0.1 0.03
inds file already exists for donwnsample of 0.100 at /Users/omar/projects/planttech/data/test_kiwi/lad_0.1/inds.npy
max --> [100, 112, 33]
min --> [0, 0, 0]


35680it [32:34, 18.26it/s]


tot vox: 	 388042
voxels hitted: 	 287106
Percentage of voxels hitted by beam: 0.74
voxels hitted (OLD): 	 316591
Percentage of voxels hitted by beam (OLD): 0.82


In [80]:
for DS in [0.2]:

    VS = ds2vs(DS, 0.014)
    VS = np.round(VS, 3)

    runall(downsample=DS, voxel_size=VS, kbins=3)

downsample: 0.2
voxel_size: 0.024
max --> [125, 141, 42]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (126, 142, 43)
ray tracker voxel dimensions: 	 (126, 142, 43)
Number of voxels ocupied by points cloud: 	 18343
Number of voxels ocupied by beam points cloud: 	 588106
Total number of voxels in plant regions: 	 769356
Number of voxels with attribute 1: 	 18343
Number of voxels with attribute 2: 	 588106
Number of voxels with attribute 3: 	 162907
max --> [125, 141, 42]
min --> [0, 0, 0]
18343 7345 10998
Counts 29756
Density overal= 2797.78
Mean density= 112776.50
Mean density considering all voxels= 2644.99
Median density= 72337.96
3


In [68]:
downsample = 0.05
voxel_size = 0.05

runall(downsample, voxel_size, kbins=2)

downsample: 0.05
voxel_size: 0.05
max --> [59, 68, 19]
min --> [0, 0, 0]
foliage voxel dimensions: 	 (60, 69, 20)
ray tracker voxel dimensions: 	 (60, 69, 20)
Number of voxels ocupied by points cloud: 	 4791
Number of voxels ocupied by beam points cloud: 	 58935
Total number of voxels in plant regions: 	 82800
Number of voxels with attribute 1: 	 4791
Number of voxels with attribute 2: 	 58935
Number of voxels with attribute 3: 	 19074
max --> [59, 68, 19]
min --> [0, 0, 0]
4791 2055 2736
Counts 8169
Density overal= 789.28
Mean density= 12833.26
Mean density considering all voxels= 723.19
Median density= 8000.00
2


In [124]:

ks = []
for v in vox:

    i,j,k = v.split('_')
    ks.append(int(k))

ks = np.array(ks)


In [129]:
counts[ks == 2].sum()

704